In [1]:
import numpy as np
import torch 

from src.neural_net import *
from cvxpy_dpc_layer import *
from trainer import *

In [2]:
bundle = np.load("data/data.npz")


In [3]:
X = bundle["X"]; Y = bundle["Y"]; U = bundle["U"]; Xtraj = bundle["Xtraj"]
OBJ = bundle["OBJ"]; status = bundle["STATUS"]

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test, U_train, U_test, Xtraj_train, Xtraj_test, OBJ_train, OBJ_test = train_test_split(
    X, Y, U, Xtraj, OBJ, test_size=0.2, random_state=42
)


In [5]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
Y_test_tensor = torch.tensor(Y_test, dtype=torch.float32)
U_train_tensor = torch.tensor(U_train, dtype=torch.float32)
U_test_tensor = torch.tensor(U_test, dtype=torch.float32)
Xtraj_train_tensor = torch.tensor(Xtraj_train, dtype=torch.float32)
Xtraj_test_tensor = torch.tensor(Xtraj_test, dtype=torch.float32)
# OBJ_train_tensor = torch.tensor(OBJ_train, dtype=torch.float32)
# OBJ_test_tensor = torch.tensor(OBJ_test, dtype=torch.float32)

train_dataset = torch.utils.data.TensorDataset(
    X_train_tensor, Y_train_tensor, Xtraj_train_tensor, U_train_tensor
)
test_dataset = torch.utils.data.TensorDataset(
    X_test_tensor, Y_test_tensor, Xtraj_test_tensor, U_test_tensor
)

In [6]:
batch_size = 128
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False
)

In [7]:
cp_layer = build_dpc_cvxpy_layer(N=20, use_soft=False)

/home/x_lab/workspace/l2o/SS_Learning_MIQP_1/.venv/lib/python3.12/site-packages/cvxpy/expressions/expression.py:498: FutureWarning: 
    You didn't specify the order of the flatten expression. The default order
    used in CVXPY is Fortran ('F') order. This default will change to match NumPy's
    default order ('C') in a future version of CVXPY.
    To suppress this warning, please specify the order explicitly.
    
  warnings.warn(flatten_order_warning, FutureWarning)


In [8]:
# def _obj_function(u_opt, x_opt, y, meta=None):
#     """
#     Example objective function computation.
#     Args:
#         u_opt: Optimal control inputs (B, H, 2)
#         x_opt: Optimal states (B, H+1, 2)
#         y: integer variables (B, H) - delta values in {0,1,2,3}
#         meta: Dictionary containing objective parameters
#     Returns:
#         Objective value tensor (B,).
#     """
#     device = u_opt.device
#     B, H, _ = u_opt.shape
    
#     # Default weights (from data_collection.py)
#     if meta is None:
#         Q = torch.eye(2, device=device)           # state cost
#         R = torch.eye(2, device=device) * 0.5           # control cost
#         P = torch.eye(2, device=device)           # terminal cost
#         rho = torch.tensor(0.1, device=device)    # integer variable penalty
#         x_ref = torch.tensor([4.2, 1.8], device=device).view(1, 1, 2).expand(B, H+1, 2)
#     else:
#         Q = meta.get('Q', torch.eye(2)).to(device)
#         R = meta.get('R', torch.eye(2)).to(device)
#         P = meta.get('P', torch.eye(2)).to(device)
#         rho = meta.get('rho', torch.tensor(0.1)).to(device)
#         x_ref = meta.get('x_ref', torch.tensor([4.2, 1.8], device=device).view(1, 1, 2).expand(B, H+1, 2)).to(device)
    
#     # Stage costs over horizon (k = 0 to H-1)
#     # State cost: sum_k (x_k - x_ref)^T Q (x_k - x_ref)
#     state_error = x_opt[:, :-1, :] - x_ref[:, :-1, :]  # (B, H, 2) - exclude terminal state
#     state_cost = torch.einsum('bhi,ij,bhj->b', state_error, Q, state_error)
    
#     # Control cost: sum_k u_k^T R u_k
#     control_cost = torch.einsum('bhi,ij,bhj->b', u_opt, R, u_opt)
    
#     # Integer variable penalty: sum_k rho * delta_k^2
#     if y.dim() == 2:  # (B, H)
#         integer_penalty = rho * (y ** 2).sum(dim=1)  # (B,)
#     elif y.dim() == 3:  # (B, H, 1) or similar
#         integer_penalty = rho * (y ** 2).sum(dim=[1, 2])  # (B,)
#     else:
#         integer_penalty = torch.zeros(B, device=device)

#     # Terminal cost: (x_N - x_ref)^T P (x_N - x_ref)
#     terminal_error = x_opt[:, -1, :] - x_ref[:, -1, :]  # (B, 2)
#     terminal_cost = torch.einsum('bi,ij,bj->b', terminal_error, P, terminal_error)
    
#     # Total objective
#     obj = state_cost + control_cost + integer_penalty + terminal_cost  # (B,)
#     return obj

In [9]:
# for x, y, u, x_traj, obj in train_loader:
#     print(x.shape, y.shape)
#     x0 = x[:, 0:2]
#     d = x[:, 2:].reshape(-1, 20, 2)

#     print(y.shape)

#     x_opt, u_opt, s_opt = cp_layer(x0, d, y)

#     # print(x.shape)
#     print(x_opt.shape, u_opt.shape)
#     print(x_traj.shape, u.shape)

#     threshold = 1e-3
#     diff_x = x_opt - x_traj
#     diff_u = u_opt - u
    
#     print("X trajectory differences (ignoring < threshold):")
#     print(diff_x[torch.abs(diff_x) >= threshold])
    
#     print("\nControl input differences (ignoring < threshold):")
#     print(diff_u[torch.abs(diff_u) >= threshold])

#     # print(torch.mean(_obj_function(u_opt, x_opt, y) - obj))
#     print(torch.mean(_obj_function(u, x_traj, y) - obj))
#     print(torch.mean(_obj_function(u, x_traj, y) - torch.mean(_obj_function(u_opt, x_opt, y))))
#     # print()

#     break

In [10]:
nn_model_0 = MLPWithSoftmaxSTE(
    insize=42,
    outsize=20,
    integer_choices=[0, 1, 2, 3],
    hsizes=[128] * 4
)


In [11]:
import datetime
now = datetime.datetime.now()
dt_string = now.strftime("%Y%m%d_%H%M%S")

In [12]:
training_params = {}
training_params['TRAINING_EPOCHS'] = int(10)
training_params['CHECKPOINT_AFTER'] = int(50)
training_params['LEARNING_RATE'] = 1e-3
training_params['WEIGHT_DECAY'] = 1e-5
training_params['PATIENCE'] = 5
training_params['WANDB_PROJECT'] = "l2o_ssl_miqp_dpc"
training_params['RUN_NAME'] = "sl_" + dt_string


Trainer_SL = SSL_MIQPP_Trainer(
    nn_model=nn_model_0,
    cvx_layer=cp_layer,
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
)

Trainer_SL.train_SL(
    train_loader=train_loader,
    test_loader=test_loader,
    training_params=training_params,
    wandb_log=True
)

wandb: Currently logged in as: mux2001 (mux2001-xxxxxlab-test1) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


[epoch 1 | step 1] training loss = 0.9078, validation loss = 0.9269
[epoch 1 | step 50] training loss = 0.3990, validation loss = 0.3622
[epoch 2 | step 100] training loss = 0.2662, validation loss = 0.2458
[epoch 3 | step 150] training loss = 0.2170, validation loss = 0.2057
[epoch 4 | step 200] training loss = 0.1627, validation loss = 0.1802
[epoch 4 | step 250] training loss = 0.1807, validation loss = 0.1649
[epoch 5 | step 300] training loss = 0.1643, validation loss = 0.1559
[epoch 6 | step 350] training loss = 0.1570, validation loss = 0.1532
[epoch 7 | step 400] training loss = 0.1346, validation loss = 0.1444
[epoch 8 | step 450] training loss = 0.1490, validation loss = 0.1420
[epoch 8 | step 500] training loss = 0.1314, validation loss = 0.1328
[epoch 9 | step 550] training loss = 0.1238, validation loss = 0.1349


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


[epoch 10 | step 600] training loss = 0.1342, validation loss = 0.1296


epoch,▁▁▂▃▃▃▄▅▆▆▆▇█
step,▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▄▄▅▅▆▆▆▆▆▇▇▇▇▇████
train/loss,█▆▅▅▄▃▂▂▂▂▂▂▂▂▂▁▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁
val/loss,█▃▂▂▁▁▁▁▁▁▁▁▁
epoch,10
step,629
train/loss,0.13945
val/loss,0.12962


In [13]:
Trainer_SL.evaluate(test_loader)

Evaluation Results:
avg_obj_val: 636.9917
avg_optimality_gap: 334.7316
avg_slack_penalty: 0.0494
avg_y_penalty: 0.0000
avg_supervised_loss: 0.1265


{'avg_obj_val': 636.99169921875,
 'avg_optimality_gap': 334.73162841796875,
 'avg_slack_penalty': 0.049377962946891785,
 'avg_y_penalty': 0.0,
 'avg_supervised_loss': 0.12654297053813934}

In [14]:
nn_model = MLPWithSoftmaxSTE(
    insize=42,
    outsize=20,
    integer_choices=[0, 1, 2, 3],
    hsizes=[128] * 4
)

Trainer_SSL = SSL_MIQPP_Trainer(
    nn_model=nn_model,
    cvx_layer=cp_layer,
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
)


slack_weight = 1e5
constraint_weight = 0
supervised_weight = 1e5
loss_weights = [1.0, slack_weight, constraint_weight, supervised_weight]

Trainer_SSL.train_SSL(
    train_loader=train_loader,
    test_loader=test_loader,
    training_params=training_params,
    loss_weights=loss_weights,
    wandb_log=True
)

[epoch 1 | step 1] validation: loss = 0.5985, obj_val = 2720.9929, opt_gap = 2103.8789 %, slack_pen = 0.2574, y_penalty = 0.0000, supervised_loss = 0.9123, 
[epoch 1 | step 50] validation: loss = 0.1996, obj_val = 617.8381, opt_gap = 400.4204 %, slack_pen = 0.0490, y_penalty = 0.0000, supervised_loss = 0.3441, 
[epoch 2 | step 100] validation: loss = 0.1420, obj_val = 378.4980, opt_gap = 206.5659 %, slack_pen = 0.0254, y_penalty = 0.0000, supervised_loss = 0.2549, 
[epoch 3 | step 150] validation: loss = 0.1075, obj_val = 326.8928, opt_gap = 164.7681 %, slack_pen = 0.0204, y_penalty = 0.0000, supervised_loss = 0.1914, 
[epoch 4 | step 200] validation: loss = 0.1050, obj_val = 314.0634, opt_gap = 154.3768 %, slack_pen = 0.0191, y_penalty = 0.0000, supervised_loss = 0.1877, 
[epoch 4 | step 250] validation: loss = 0.0888, obj_val = 309.6043, opt_gap = 150.7652 %, slack_pen = 0.0187, y_penalty = 0.0000, supervised_loss = 0.1559, 
[epoch 5 | step 300] validation: loss = 0.0900, obj_val = 3

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


epoch,▁▁▂▃▃▃▄▅▆▆▆▇█
step,▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇█
train/combined_loss,█▆▃▃▂▂▂▂▂▂▂▁▂▁▂▁▂▁▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁
train/obj_val,█▄▃▂▃▂▂▂▂▂▃▂▄▂▂▁▂▃▅▃▁▂▁▂▃▁▁▂▂▇▅▄▂▁▁▃▂▂▂▂
train/slack_pen,█▆▃▂▂▂▂▂▁▁▂▁▅▁▁▂▃▂▄▃▃▂▁▁▁▁▂▅▂▂▂▂▁▂▂▂▂▃▂▂
train/supervised_loss,█▇▆▃▃▃▃▃▂▂▂▂▁▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/y_penalty,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/avg_loss,█▃▂▁▁▁▁▁▁▁▁▁▁
val/obj_val,█▂▁▁▁▁▁▁▁▁▁▁▁
val/slack_pen,█▂▁▁▁▁▁▁▁▁▁▁▁
+2,...


In [15]:
Trainer_SSL.evaluate(test_loader)

Evaluation Results:
avg_obj_val: 620.1743
avg_optimality_gap: 322.8466
avg_slack_penalty: 0.0477
avg_y_penalty: 0.0000
avg_supervised_loss: 0.1291


{'avg_obj_val': 620.1742553710938,
 'avg_optimality_gap': 322.8466491699219,
 'avg_slack_penalty': 0.04769348353147507,
 'avg_y_penalty': 0.0,
 'avg_supervised_loss': 0.12907715141773224}